### This notebook will only work with th data which already clean by the EDA notebook.

In [27]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split


In [14]:
ebay_clean= pd.read_csv('../data/cleaned/Ebay_cleaned.csv', index_col=0)
pd.set_option('display.max_columns', None)
ebay_clean.head()

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight_units,package_size,record_number,distance,handling_date,shipping_date,total_time
0,1,25454,3.0,2019-03-26 15:11:00,0,0.0,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49,2019-03-29,1,1.0,1,3001.839179,2,2,5
1,0,6727381,2.0,2018-06-02 12:53:00,0,3.0,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54,2018-06-05,1,4.0,2,1282.908017,1,2,3
2,1,18507,1.0,2019-01-07 16:22:00,0,4.5,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00,2019-01-10,1,4.0,3,1104.444168,2,2,4
3,1,4677,1.0,2018-12-17 16:56:00,0,0.0,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28,2018-12-21,1,4.0,4,1353.390003,1,3,5
4,1,4677,1.0,2018-07-27 16:48:00,0,0.0,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02,2018-07-30,1,4.0,5,2456.330752,1,2,3


In [15]:
ebay_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93785 entries, 0 to 99999
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   b2c_c2c                    93785 non-null  int64  
 1   seller_id                  93785 non-null  int64  
 2   declared_handling_days     93785 non-null  float64
 3   acceptance_scan_timestamp  93785 non-null  object 
 4   shipment_method_id         93785 non-null  int64  
 5   shipping_fee               93785 non-null  float64
 6   carrier_min_estimate       93785 non-null  int64  
 7   carrier_max_estimate       93785 non-null  int64  
 8   item_zip                   93785 non-null  object 
 9   buyer_zip                  93785 non-null  object 
 10  category_id                93785 non-null  int64  
 11  item_price                 93785 non-null  float64
 12  quantity                   93785 non-null  int64  
 13  payment_datetime           93785 non-null  obj

In [16]:
X= ebay_clean.drop(columns=['total_time','payment_datetime', 'delivery_date', 'acceptance_scan_timestamp', 'item_zip', 'buyer_zip'])
y= ebay_clean['total_time']

In [18]:
print(y.shape)
print(X.shape)

(93785,)
(93785, 16)


In [23]:
#split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#### LinearRegression model

In [28]:
#Initialize
linear_model= LinearRegression()
#fit model
linear_model.fit(X_train, y_train)
linear_preds= linear_model.predict(X_test)
linear_score_test=  linear_model.score(X_test, y_test)
linear_score_train=  linear_model.score(X_train, y_train)

In [25]:
linear_preds

array([2.24252004, 3.14047095, 3.10482214, ..., 3.10992504, 4.08845012,
       2.02110675])

In [29]:
print(f'linear_score_test: {linear_score_test}')
print(f'linear_score_train: {linear_score_train}')

linear_score_test: 0.9683468853522164
linear_score_train: 0.9691357452376683


#### Ridge model

In [32]:
# Initialize
ridge_model= Ridge(alpha= 0.5)
# fit
ridge_model.fit(X_train, y_train)

rigde_preds= ridge_model.predict(X_test)
ridge_score_test= ridge_model.score(X_test, y_test)
ridge_score_train= ridge_model.score(X_train, y_train)

/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.45693e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [33]:
print(f"train: {ridge_score_train} -- test: {ridge_score_test} ")

train: 0.9691357452350413 -- test: 0.968346876061139 
